## Imports and setting up db connection

In [2]:
import yfinance as yf
import pandas as pd
import sqlite3

from data import Data

data = Data()

symbols = ["SPY", "IEF", "TLT", "GLD", "VNQ"]

## Update db data

In [2]:
data.updateEodData()

[*********************100%***********************]  1 of 1 completed
SPY is already up to date (latest date: 2020-06-12)
[*********************100%***********************]  1 of 1 completed
IEF is already up to date (latest date: 2020-06-12)
[*********************100%***********************]  1 of 1 completed
TLT is already up to date (latest date: 2020-06-12)
[*********************100%***********************]  1 of 1 completed
GLD is already up to date (latest date: 2020-06-12)
[*********************100%***********************]  1 of 1 completed
VNQ is already up to date (latest date: 2020-06-12)


## Reading data from the db

In [3]:
data.getEodData("TLT", start="2019-04-03", end="2020-01-01")

,open,high,low,close,adj_close,volume
date,,,,,,
2019-04-03,123.61,123.87,123.37,123.52,120.6112,10407000
2019-04-04,123.60,123.89,123.44,123.87,120.9530,7503400
2019-04-05,123.62,124.17,123.58,124.03,121.1092,6321800
2019-04-08,123.83,123.95,123.52,123.59,120.6796,4446400
2019-04-09,124.22,124.34,123.77,123.98,121.0604,7020300
...,...,...,...,...,...,...
2019-12-24,135.92,137.06,135.89,136.84,135.8637,4667400
2019-12-26,136.91,137.22,136.51,137.17,136.1913,5290200
2019-12-27,137.44,137.63,137.29,137.32,136.3403,7300500


## Web scraping

In [28]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

url = "https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average"
response = requests.get(url)
response.raise_for_status()

soup = BeautifulSoup(response.text, 'html.parser')
table = soup.find(id="constituents")
headers = list(map(lambda x: x.get_text(strip=True), table.find_all("th")))
table_rows = table.find("tbody").find_all("tr")

temp = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.get_text(strip=True) for tr in td]
    temp.append(row)

df = pd.DataFrame(temp[1:], columns=headers)
df



,Company,Exchange,Symbol,Industry,Date Added,Notes,"Index Weighting (Apr 30, 2020)"
0,3M,NYSE,NYSE:MMM,Conglomerate,1976-08-09,as Minnesota Mining and Manufacturing,4.35%
1,American Express,NYSE,NYSE:AXP,Financial services,1982-08-30,,2.68%
2,Apple Inc.,NASDAQ,AAPL,Information technology,2015-03-19,,8.01%
3,Boeing,NYSE,NYSE:BA,Aerospaceanddefense,1987-03-12,,3.87%
4,Caterpillar Inc.,NYSE,NYSE:CAT,ConstructionandMining,1991-05-06,,3.34%
5,Chevron Corporation,NYSE,NYSE:CVX,Petroleum industry,2008-02-19,also 1930-07-18 to 1999-11-01,2.63%
6,Cisco Systems,NASDAQ,CSCO,Information technology,2009-06-08,,1.21%
7,The Coca-Cola Company,NYSE,NYSE:KO,Food industry,1987-03-12,also 1932-05-26 to 1935-11-20,1.31%
8,Dow Inc.,NYSE,NYSE:DOW,Chemical industry,2019-04-02,,1.04%
9,ExxonMobil,NYSE,NYSE:XOM,Petroleum industry,1928-10-01,asStandard Oil of New Jersey,1.32%


## Formatting data to put into database

In [29]:
extraColumns = list(filter(lambda x: x not in ["Company", "Exchange", "Symbol"], headers))

df.drop(labels=extraColumns, axis=1, inplace=True)

df['Symbol'] = df['Symbol'].apply(lambda x: x[x.find(":") + 1:])
df['Exchange'] = df['Exchange'].apply(lambda x: data.getExchangeId(x))

df.rename(
    columns={
        "Exchange": "exchange_id",
        "Symbol": "symbol",
        "Company": "name"
    },
    inplace=True)

df.insert(loc=len(df.columns), column="currency", value="USD")
df.insert(loc=len(df.columns), column="instrument", value="Stock")

df

,name,exchange_id,symbol,currency,instrument
0,3M,2,MMM,USD,Stock
1,American Express,2,AXP,USD,Stock
2,Apple Inc.,3,AAPL,USD,Stock
3,Boeing,2,BA,USD,Stock
4,Caterpillar Inc.,2,CAT,USD,Stock
5,Chevron Corporation,2,CVX,USD,Stock
6,Cisco Systems,3,CSCO,USD,Stock
7,The Coca-Cola Company,2,KO,USD,Stock
8,Dow Inc.,2,DOW,USD,Stock
9,ExxonMobil,2,XOM,USD,Stock


In [31]:
# df.to_sql('assets', data.con, index=False, if_exists="append")